<a href="https://colab.research.google.com/github/Priyanka-704/BNPL-Risk-Analysis-Fintech-Project/blob/main/Day0_Setup_BNPL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install libraries for BNPL Risk Analysis (most are pre-installed in Colab)
!pip install --quiet pandas numpy scikit-learn plotly pandas-datareader
# sqlite3 is built-in, no install needed
print("✅ All libraries installed successfully!")
print("Ready for Python/SQL in Fintech project.")

✅ All libraries installed successfully!
Ready for Python/SQL in Fintech project.


In [4]:
# Test imports and versions for BNPL project
import pandas as pd
import numpy as np
import sqlite3
from sklearn.linear_model import LogisticRegression  # For future ML
import plotly.express as px  # For dashboards
import pandas_datareader as pdr  # Optional for economic data

print("✅ Python Environment Test Passed!")
print(f"Pandas version: {pd.__version__} (for data manipulation)")
print(f"NumPy version: {np.__version__} (for calculations)")
print("SQLite3: Connected (for BNPL database)")
print("Scikit-learn: Ready for risk modeling")
print("Plotly: Ready for interactive visuals")
print("Pandas-datareader: Ready for API pulls (e.g., economic risks)")

# Quick SQLite connection test (no data yet)
conn = sqlite3.connect(':memory:')  # In-memory DB for test
conn.execute("CREATE TABLE test (id INTEGER PRIMARY KEY)")
conn.close()
print("✅ SQLite test: Table created successfully!")

✅ Python Environment Test Passed!
Pandas version: 2.2.2 (for data manipulation)
NumPy version: 2.0.2 (for calculations)
SQLite3: Connected (for BNPL database)
Scikit-learn: Ready for risk modeling
Plotly: Ready for interactive visuals
Pandas-datareader: Ready for API pulls (e.g., economic risks)
✅ SQLite test: Table created successfully!


In [5]:
#step 3 mock data test
import pandas as pd
import numpy as np

# Mock BNPL data: 5 sample user loans (expand to real datasets later)
# Columns: user_id, loan_amount (purchase value), due_date, status (paid/overdue/default), purchase_category
mock_data = {
    'user_id': [1, 1, 2, 3, 4],  # User 1 has two loans
    'loan_amount': [100, 150, 200, 50, 300],  # Small BNPL-style purchases
    'due_date': ['2023-10-01', '2023-10-15', '2023-11-01', '2023-09-20', '2023-10-10'],
    'status': ['paid', 'overdue', 'paid', 'default', 'paid'],  # Key for risk: 1 default, 1 overdue
    'purchase_category': ['electronics', 'fashion', 'groceries', 'fashion', 'electronics']
}
df = pd.DataFrame(mock_data)

# Display the data
print("✅ Mock BNPL Data Created (5 rows for testing):")
print(df)

# Quick insight: Calculate sample default rate (business metric for risk)
default_rate = (df['status'] == 'default').sum() / len(df) * 100
overdue_rate = (df['status'] == 'overdue').sum() / len(df) * 100
print(f"\n📊 Sample Insights:")
print(f"Default Rate: {default_rate:.1f}% (high-risk indicator)")
print(f"Overdue Rate: {overdue_rate:.1f}% (early warning for defaults)")

✅ Mock BNPL Data Created (5 rows for testing):
   user_id  loan_amount    due_date   status purchase_category
0        1          100  2023-10-01     paid       electronics
1        1          150  2023-10-15  overdue           fashion
2        2          200  2023-11-01     paid         groceries
3        3           50  2023-09-20  default           fashion
4        4          300  2023-10-10     paid       electronics

📊 Sample Insights:
Default Rate: 20.0% (high-risk indicator)
Overdue Rate: 20.0% (early warning for defaults)


In [6]:
#Connect to SQLite and Insert/Query the Data
import sqlite3

# Step 1: Connect to SQLite DB (creates 'bnpl_risk.db' file)
conn = sqlite3.connect('bnpl_risk.db')
print("✅ Connected to SQLite DB for BNPL data.")

# Step 2: Insert the DataFrame into a table
df.to_sql('bnpl_loans', conn, if_exists='replace', index=False)
print("✅ Data inserted into 'bnpl_loans' table.")

# Step 3: Basic SQL Query - Fetch all overdue/default loans
query = """
SELECT user_id, loan_amount, status, purchase_category
FROM bnpl_loans
WHERE status IN ('overdue', 'default')
ORDER BY loan_amount DESC
"""
risky_loans = pd.read_sql_query(query, conn)
print("\n📊 SQL Query Results: Risky Loans (Overdue/Default):")
print(risky_loans)

# Step 4: Aggregate query - Risk by category
agg_query = """
SELECT purchase_category,
       COUNT(*) as num_loans,
       SUM(CASE WHEN status = 'default' THEN 1 ELSE 0 END) as defaults,
       ROUND(AVG(loan_amount), 2) as avg_amount
FROM bnpl_loans
GROUP BY purchase_category
"""
category_risk = pd.read_sql_query(agg_query, conn)
print("\n📊 Aggregated Risk by Category:")
print(category_risk)

# Close connection
conn.close()
print("\n✅ DB operations complete! File 'bnpl_risk.db' saved.")

✅ Connected to SQLite DB for BNPL data.
✅ Data inserted into 'bnpl_loans' table.

📊 SQL Query Results: Risky Loans (Overdue/Default):
   user_id  loan_amount   status purchase_category
0        1          150  overdue           fashion
1        3           50  default           fashion

📊 Aggregated Risk by Category:
  purchase_category  num_loans  defaults  avg_amount
0       electronics          2         0       200.0
1           fashion          2         1       100.0
2         groceries          1         0       200.0

✅ DB operations complete! File 'bnpl_risk.db' saved.


In [7]:
# Reconnect to DB to add risk flag (or work on DataFrame)
conn = sqlite3.connect('bnpl_risk.db')
df_with_risk = pd.read_sql_query("SELECT * FROM bnpl_loans", conn)

# Simple rule-based risk flag (teaser for Phase 3 modeling)
df_with_risk['risk_flag'] = np.where(
    df_with_risk['status'].isin(['overdue', 'default']), 'High Risk',
    np.where(df_with_risk['loan_amount'] > 200, 'Medium Risk', 'Low Risk')
)
df_with_risk['risk_score'] = np.where(df_with_risk['risk_flag'] == 'High Risk', 80,
                                      np.where(df_with_risk['risk_flag'] == 'Medium Risk', 50, 20))

# Save enhanced data back (optional for now)
df_with_risk.to_sql('bnpl_loans_enhanced', conn, if_exists='replace', index=False)

print("✅ Enhanced Data with Risk Flags:")
print(df_with_risk[['user_id', 'status', 'loan_amount', 'risk_flag', 'risk_score']])

conn.close()
print("\n💡 Project Teaser: User 3 has High Risk (default) – we'll predict more in Phase 3!")

✅ Enhanced Data with Risk Flags:
   user_id   status  loan_amount    risk_flag  risk_score
0        1     paid          100     Low Risk          20
1        1  overdue          150    High Risk          80
2        2     paid          200     Low Risk          20
3        3  default           50    High Risk          80
4        4     paid          300  Medium Risk          50

💡 Project Teaser: User 3 has High Risk (default) – we'll predict more in Phase 3!


In [8]:
from google.colab import files
files.download('bnpl_risk.db')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>